In [1]:
from data.germeval2017 import germeval2017_dataset

from misc.preferences import PREFERENCES
from misc.visualizer import *
from misc.hyperparameters import get_default_params
from optimizer import get_default_optimizer
from misc import utils
from models.transformer.encoder import TransformerEncoder
from models.softmax_output import SoftmaxOutputLayerWithCommentWiseClass
from models.transformer_tagger import TransformerTagger
from models.transformer.train import Trainer
from criterion import NllLoss

C:\Users\felix\Anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
experiment_name = 'generalSentimentStdModel5Ep'

In [3]:
PREFERENCES.defaults(
    data_root='./data/germeval2017',
    data_train='train_v1.4.tsv',    
    data_validation='dev_v1.4.tsv',
    data_test='test_TIMESTAMP1.tsv',
    early_stopping='highest_5_F1'
)

hyper_parameters = get_default_params()
hyper_parameters.model_size = 300
hyper_parameters.batch_size = 80
hyper_parameters.early_stopping = -1
experiment_name = utils.create_loggers(experiment_name=experiment_name)

Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\just_testing


In [4]:
conll2003 = conll2003_dataset('ner', hyper_parameters.batch_size,
                              root=PREFERENCES.data_root,
                              train_file=PREFERENCES.data_train,
                              validation_file=PREFERENCES.data_validation,
                              test_file=PREFERENCES.data_test,
                              use_cuda=True)



In [5]:
samples = extract_samples(conll2003['examples'])
print_samples(samples)

eu - I-ORG
rejects - O
german - I-MISC
call - O
to - O
boycott - O
british - I-MISC
lamb - O
. - O

#######################

peter - I-PER
blackburn - I-PER

#######################

brussels - I-LOC
1996-08-22 - O

#######################

cricket - O
- - O
leicestershire - I-ORG
take - O
over - O
at - O
top - O
after - O
innings - O
victory - O
. - O

#######################

london - I-LOC
1996-08-30 - O

#######################

west - I-MISC
indian - I-MISC
all-rounder - O
phil - I-PER
simmons - I-PER
took - O
four - O
for - O
0 - O
on - O
friday - O
as - O
leicestershire - I-ORG
beat - O
somerset - I-ORG
by - O
an - O
innings - O
and - O
0 - O
runs - O
in - O
two - O
days - O
to - O
take - O
over - O
at - O
the - O
head - O
of - O
the - O
county - O
championship - O
. - O

#######################

soccer - O
- - O
japan - I-LOC
get - O
lucky - O
win - O
, - O
china - I-PER
in - O
surprise - O
defeat - O
. - O

#######################

nadim - I-PER
ladki - I-PER

################

In [6]:
# 10 words with a 100-length embedding
target_vocab = conll2003['vocabs'][1]
target_size = len(target_vocab)
print('Targets:',target_vocab.freqs.keys())
print(target_vocab.freqs)
print(target_vocab.itos)
class_labels = target_vocab.itos

Targets: dict_keys(['I-ORG', 'O', 'I-MISC', 'I-PER', 'I-LOC', 'B-LOC', 'B-MISC', 'B-ORG'])
Counter({'O': 169578, 'I-PER': 11128, 'I-ORG': 10001, 'I-LOC': 8286, 'I-MISC': 4556, 'B-MISC': 37, 'B-ORG': 24, 'B-LOC': 11})
[' UNK ', '<pad>', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'I-MISC', 'B-MISC', 'B-ORG', 'B-LOC']


In [7]:
total_samples = 0
for l, freq in target_vocab.freqs.items():
    print('{}: {}'.format(l, freq))
    total_samples += freq
print('\nTotal Samples:',total_samples)

print('\n\nTrivial classifiers')
for l, freq in target_vocab.freqs.items():
    acc = float(freq) / float(total_samples)
    print('Trivial {}: {}%'.format(l, acc*100))

I-ORG: 10001
O: 169578
I-MISC: 4556
I-PER: 11128
I-LOC: 8286
B-LOC: 11
B-MISC: 37
B-ORG: 24

Total Samples: 203621


Trivial classifiers
Trivial I-ORG: 4.9115759180045275%
Trivial O: 83.2811939829389%
Trivial I-MISC: 2.2374902392189413%
Trivial I-PER: 5.4650551760378345%
Trivial I-LOC: 4.069324873171235%
Trivial B-LOC: 0.005402193290475934%
Trivial B-MISC: 0.01817101379523723%
Trivial B-ORG: 0.011786603542856582%


In [8]:
loss = NllLoss(target_size)
# transformer = GoogleTransformer(True, target_size, target_size, num_units, 2, 2, 512, 0.1)
transformer = TransformerEncoder(conll2003['embeddings'][0],
                                 n_enc_blocks=2,
                                 n_head=3,
                                 d_model=hyper_parameters.model_size,
                                 d_k=100,
                                 d_v=100)
tagging_softmax = SoftmaxOutputLayer(hyper_parameters.model_size, target_size)
model = TransformerTagger(transformer, tagging_softmax)


In [9]:
# predict now to see model in final state
#df = predict_some_examples_to_df(model, conll2003['iters'][1], num_samples=400)
#print(df)

In [10]:
optimizer = get_default_optimizer(model, hyper_parameters)
trainer = Trainer(target_size, 
                    model,
                    loss,
                    optimizer,
                    hyper_parameters,
                    conll2003['iters'],
                    experiment_name,
                    log_every_xth_iteration=-1,
                    enable_tensorboard=True,
                    dummy_input=conll2003['dummy_input'])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1              [-1, 42, 300]       7,816,800
           Dropout-2              [-1, 42, 300]               0
PositionalEncoding2-3              [-1, 42, 300]               0
            Linear-4              [-1, 42, 300]          90,000
            Linear-5              [-1, 42, 300]          90,000
            Linear-6              [-1, 42, 300]          90,000
           Dropout-7               [-1, 42, 42]               0
ScaledDotProductAttentionLayer-8              [-1, 42, 100]               0
            Linear-9              [-1, 42, 300]          90,000
          Dropout-10              [-1, 42, 300]               0
        LayerNorm-11              [-1, 42, 300]               0
MultiHeadedSelfAttentionLayer-12              [-1, 42, 300]               0
           Linear-13             [-1, 42, 2048]         616,448
           Lin

In [11]:
result = trainer.train(5, True, False)

pre_training - DEBUG - train with cuda support
pre_training - INFO - 176 Iterations per epoch with batch size of 80
pre_training - INFO - START training.





A Jupyter Widget

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time
1	176	0.843		0.722		0.771		0.834		2.13m - 2.1m / 0.0m


A Jupyter Widget

...

2	352	0.114		0.383		0.880		0.909		2.15m - 4.3m / 10.7m


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
model = result['model']

In [ ]:
result_labels = trainer.classify_sentence('I was born in 1993 in Stuttgart')



print(result_labels)

In [ ]:
tr = trainer.text_reverser[1]
lr = trainer.label_reverser

test_sentence = ['china', 'controlled', 'most', 'of', 'the', 'match']
test_sentence

In [ ]:
test_sentence = tr.preprocess('My name is Felix and I was born in Germany')

#test_sentence = tr.preprocess('china controlled most of the match on 1993')
test_sentence = [x.strip(' ') for x in test_sentence]
test_sentence = [test_sentence]
test_sentence

In [ ]:

#test = tr.preprocess('china controlled most of the match')
#print(test)

#test_sentence = [['china', 'controlled', 'most', 'of', 'the', 'match']]
x = tr.process(test_sentence)

print("X TENSOR ",x)
print('X Size', x.size())
print("Reversed X", tr.reverse(x))
x = x.cuda()
y_hat = model.predict(x)
y_hat_label = lr.reverse(y_hat)
print(y_hat_label)



In [ ]:
trainer.tb_writer = None
trainer.enable_tensorboard = False
evaluation_results = trainer.perform_final_evaluation()

In [ ]:
c_matrix = evaluation_results[1][2]
c_matrix

In [ ]:
fig = plot_confusion_matrix(c_matrix, class_labels)
plt.show()

In [ ]:
fig = plot_confusion_matrix(c_matrix, class_labels, normalize=True)
plt.show()

In [ ]:
# predict now to see model in final state
from IPython.display import display, HTML

df = predict_some_examples_to_df(model, conll2003['iters'][2], num_samples=800)
display(HTML(df.to_html()))

In [ ]:
df = predict_some_examples_to_df(model, conll2003['iters'][1], num_samples=800)
display(HTML(df.to_html()))

In [ ]:
df = predict_some_examples_to_df(model, conll2003['iters'][0], num_samples=800)
display(HTML(df.to_html()))

In [ ]:
import numpy as np
a = np.array([
    np.array([[1, 1], [1, 1]]),
    np.array([[2, 2], [-2, -3]])
])
print(a)

In [ ]:
a.sum(axis=0)

In [ ]:
#df = predict_some_examples_to_df(model, test_sample_iter)
#print(df)

In [ ]:
#print(result)

In [ ]:
(tr_loss, tr_f1) = result['result_train']
